In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os
import random as rd
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')



import csv
import collections          # For OrderedDict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler



C:\Users\Donnie\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv(os.path.join("..", "Resources", "beer.csv"))
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,type,address,website,state,state_breweries
0,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
1,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
2,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
3,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156
4,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159,Microbrewery,"540 Clover Ln., Ashland, Oregon, 97520",http://www.calderabrewing.com/,oregon,156


In [3]:
df['beer_style'].nunique()

100

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361241 entries, 0 to 361240
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   brewery_id          361241 non-null  int64  
 1   brewery_name        361241 non-null  object 
 2   review_time         361241 non-null  int64  
 3   review_overall      361241 non-null  float64
 4   review_aroma        361241 non-null  float64
 5   review_appearance   361241 non-null  float64
 6   review_profilename  361241 non-null  object 
 7   beer_style          361241 non-null  object 
 8   review_palate       361241 non-null  float64
 9   review_taste        361241 non-null  float64
 10  beer_name           361241 non-null  object 
 11  beer_abv            361241 non-null  float64
 12  beer_beerid         361241 non-null  int64  
 13  type                361241 non-null  object 
 14  address             361241 non-null  object 
 15  website             361241 non-nul

In [5]:
target = df["review_overall"]

target.value_counts()





4.0    142517
4.5     82365
3.5     66898
3.0     31370
5.0     20775
2.5      9364
2.0      5445
1.5      1434
1.0      1073
Name: review_overall, dtype: int64

In [6]:
beerme = df.loc[:, ['beer_name', 'review_aroma','review_appearance','review_palate','review_taste']]

beerme.head()


,beer_name,review_aroma,review_appearance,review_palate,review_taste
0,Cauldron DIPA,4.5,4.0,4.0,4.5
1,Caldera Ginger Beer,3.5,3.5,3.0,3.5
2,Caldera Ginger Beer,3.5,3.5,4.0,4.0
3,Caldera Ginger Beer,2.5,3.5,2.0,3.5
4,Caldera Ginger Beer,3.0,3.5,3.5,4.0


In [7]:
beerreview = df.loc[:,['beer_name', 'review_overall']]

beerreview.head()

,beer_name,review_overall
0,Cauldron DIPA,4.0
1,Caldera Ginger Beer,3.0
2,Caldera Ginger Beer,3.5
3,Caldera Ginger Beer,3.0
4,Caldera Ginger Beer,4.0


In [8]:
df.iloc[:,4].value_counts()

4.0    141229
3.5     85863
4.5     66263
3.0     38878
5.0     13232
2.5      9610
2.0      4877
1.5       850
1.0       439
Name: review_aroma, dtype: int64

In [9]:
beeraroma = df.loc[:,['beer_name', 'review_aroma']]


beeraroma.head()



,beer_name,review_aroma
0,Cauldron DIPA,4.5
1,Caldera Ginger Beer,3.5
2,Caldera Ginger Beer,3.5
3,Caldera Ginger Beer,2.5
4,Caldera Ginger Beer,3.0


In [10]:
beerappearance = df.loc[:,['beer_name', 'review_appearance']]

beerappearance.head()

,beer_name,review_appearance
0,Cauldron DIPA,4.0
1,Caldera Ginger Beer,3.5
2,Caldera Ginger Beer,3.5
3,Caldera Ginger Beer,3.5
4,Caldera Ginger Beer,3.5


In [11]:
beerpalate = df.loc[:,['beer_name', 'review_palate']]

beerpalate.head()

,beer_name,review_palate
0,Cauldron DIPA,4.0
1,Caldera Ginger Beer,3.0
2,Caldera Ginger Beer,4.0
3,Caldera Ginger Beer,2.0
4,Caldera Ginger Beer,3.5


In [12]:
beertaste = df.loc[:,['beer_name', 'review_taste']]

beertaste.head()

,beer_name,review_taste
0,Cauldron DIPA,4.5
1,Caldera Ginger Beer,3.5
2,Caldera Ginger Beer,4.0
3,Caldera Ginger Beer,3.5
4,Caldera Ginger Beer,4.0


In [ ]:
beerplot = beerreview[['beer_name','review_overall']]
#Visualise data points
plt.scatter(beerplot["beer_name"],beerplot["review_overall"],c='black')
plt.xlabel('Beer')
plt.ylabel('Review')
plt.show()

In [ ]:
aromaplot = beeraroma[['beer_name','review_aroma']]
#Visualise data points
plt.scatter(beeraroma["beer_name"],beerme["review_aroma"],c='black')
plt.xlabel('Beer')
plt.ylabel('Review')
plt.show()

In [ ]:
aromaplot = beerme[['beer_name','review_appearance']]
#Visualise data points
plt.scatter(beerme["beer_name"],beerme["review_appearance"],c='black')
plt.xlabel('Beer')
plt.ylabel('Review')
plt.show()

In [ ]:
aromaplot = beerme[['beer_name','review_palate']]
#Visualise data points
plt.scatter(beerme["beer_name"],beerme["review_palate"],c='black')
plt.xlabel('Beer')
plt.ylabel('Review')
plt.show()

In [ ]:
aromaplot = beerme[['beer_name','review_taste']]
#Visualise data points
plt.scatter(beerme["beer_name"],beerme["review_taste"],c='black')
plt.xlabel('Beer')
plt.ylabel('Review')
plt.show()

In [13]:

aromamodel = {
    'modelname': 'aromamodel',
    'column': 'review_aroma',
    'numclusters': 4
}

appearancemodel = {
    'modelname': 'appearancemodel',
    'column': 'review_appearance',
    'numclusters': 4
    
}

palatemodel = {
    'modelname': 'palatemodel',
    'column': 'review_palate',
    'numclusters': 4
}

tastemodel = {
    'modelname': 'tastemodel',
    'column': 'review_taste',
    'numclusters': 4
}



In [21]:
models = [aromamodel, appearancemodel, palatemodel, tastemodel]
y = df.loc[:, 'beer_name']

for model in models:

    X = np.array(df.loc[:, aromamodel['column']])
    X = X.reshape(-1, 1)
# X.shape
    km = KMeans(n_clusters=4, init='k-means++', max_iter=1000, n_init=1, verbose=2)

    km.fit(X)


Initialization complete
Iteration 0, inertia 14256.0
Iteration 1, inertia 11142.581862995354
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 33529.25
Iteration 1, inertia 22515.7817460295
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 14256.0
Iteration 1, inertia 11142.581862995354
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 16081.5
Iteration 1, inertia 11517.106022576654
Converged at iteration 1: strict convergence.


In [ ]:
from sklearn.model_selection import train_test_split
target_train, target_test, aroma_train, aroma_test = train_test_split(target, aroma, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

aroma_scaler = StandardScaler().fit(aroma_train)

In [ ]:
target.replace([3.5,3],"C",inplace=True)
target.replace([4.5,5],"A",inplace=True)
target.replace([2.5,2,1.5,1],"D",inplace=True)
target.replace([4],"B",inplace=True)

In [ ]:
unused = df.drop(["review_overall","website","address","brewery_name", "brewery_id", "review_time","review_profilename"], axis=1)
unused.head()

In [ ]:
target = df["review_overall"]
target_names = ["negative", "positive"]